In [103]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy import text


In [104]:
# Example URLs

url = "https://www.eversports.de/s/poda-studio"

# "https://www.eversports.de/s/yoga-and-pole-art-by-selina"
# https://www.eversports.de/s/schoenheitstanz-studio

In [105]:
# Send a GET request to the webpage

response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.content, 'html.parser')

Container Segment 1 | Pole Studio Name, 

In [106]:
# Container Overview
overview_description = soup.find_all('div', class_="css-ptk251")


In [107]:
# Loop through each element in the container
for item in overview_description:
    # Pole Studio Name
    pole_studio_name = item.find('p', class_='MuiTypography-root MuiTypography-body1 css-1yc2rip').text
    print(pole_studio_name)

    # Address
    location = item.find('p', class_="MuiTypography-root MuiTypography-body1 css-e8od91").text
    print(location)


Yoga and Pole Art by Selina
Zum Wingert 10 , 53783 Eitorf


Container Segment 2 Description

In [108]:
# Container Description Pole Studio
description = soup.find_all('div', class_="css-1821gv5")


In [109]:
# Loop through each element in the container
for item in description:
    # Pole Describtion Name
    pole_studio_description = item.find('p', class_='MuiTypography-root MuiTypography-body1 css-pxnkv9').text
    print(pole_studio_description)
    if pole_studio_description.strip():
        pole_studio_description = pole_studio_description.strip()
        break

Yoga and Pole Art by Selina bietet dir Online Live Klassen (inkl. 24h Aufzeichnungen), Intensivkurse, Workshops und Personal Training in folgenden Bereichen an:
· Hatha Yoga
· Vinyasa Yoga
· Yoga Rad (auch bekannt als Yoga Wheel) für Einsteiger und Fortgeschrittene
· Aerial Yoga
· Pole Trick Training
· Spinning Pole Combos Einsteiger
· Spinning Pole Combos Intermediate
· Splits on Pole Intermediate
· Flexi Pole Tricks
· Krafttraining und Trainingsperiodisierung
· Krafttraining an der Stange
Zudem bietet dir Yoga and Pole Art by Selina zahlreiche Aufzeichnungen und Video Kurse im Pole Sport und Yoga Bereich.


Container Segment 3 Prices

In [110]:
# Container Prices
starting_price_ = soup.find_all('div', class_="css-1vt08d7")
list_5 = soup.find_all('div', class_="css-sge262")

In [111]:
# Loop through each element in the container
for item in starting_price_:
    # Starting Price
    starting_price = item.find('p', class_='MuiTypography-root MuiTypography-body1 css-13ps6ou').text
    print(starting_price)

    # Taster Course Price
    taster_course_price = item.find('p', class_="MuiTypography-root MuiTypography-body1 css-13ps6ou").text
    print(taster_course_price)


24,00 €
24,00 €


Segment 5 Rating

In [112]:
div_container = soup.find('div', class_='css-1oqii6')
div_container.find_all('p')
p_elements = div_container.find_all('p')

In [113]:
# Find the container for the rating information
rating_container = soup.find('div', class_='css-1oqii6')
ratingscore = div_container.find_all('p')[0].text
ratingcount = div_container.find_all('p')[1].text
# 
ratingfactors = [div_container.find_all('p')[2].text + ' ' + div_container.find_all('p')[3].text,
                 div_container.find_all('p')[4].text + ' ' + div_container.find_all('p')[5].text,
                 div_container.find_all('p')[6].text + ' ' + div_container.find_all('p')[7].text,
                 div_container.find_all('p')[8].text + ' ' + div_container.find_all('p')[9].text,
                 div_container.find_all('p')[10].text + ' ' + div_container.find_all('p')[11].text
                ]
ratingfactors

['Raumqualität 5.0',
 'Freundlichkeit & Service 5.0',
 'Sauberkeit 5.0',
 'Atmosphäre 5.0',
 'Preis-Leistung 4.9']

Contact Info: Homepage, Telephone, E-Mail

In [114]:
pole_info=(soup.find_all('div', class_='css-1x2phcg'))

In [115]:
contact_info = []
for div in pole_info:
    a_tags = div.find_all('a')
    contact_row = []
    for a in a_tags:
        href = a.get('href')
        if 'mailto:' in href:
            email = href.replace('mailto:', '')
            contact_row.append(email)
        elif 'http' in href:
            homepage = href
            contact_row.append(homepage)
        else:
            phone = href.replace('tel:', '')
            contact_row.append(phone)
    contact_info.append(contact_row)

contact_df = pd.DataFrame(contact_info, columns=["E-Mail", "Homepage", "Telefon"])
contact_df


,E-Mail,Homepage,Telefon
0,yogaandpoleartbyselina@gmail.com,http://www.yogaandpoleartbyselina.com,+49 15738925807


Prepare Strings 

In [116]:
location.split(',')
address = location.split(',')[1]
town = location.split(',')[0]
plz = location.split(',')[1].split(' ')[1]
print(location,address,town,plz)

Zum Wingert 10 , 53783 Eitorf  53783 Eitorf Zum Wingert 10  53783


Create Dataframe for the data

In [117]:
# create a dictionary from the given lists
pole_studio_overview = {
    'PoleStudio': pole_studio_name,
    'PLZ': plz,
    'Stadt': town,
    'Adresse': address,
    'Pole Studio Beschreibung': pole_studio_description,
    'Start Preis': starting_price,
    'Schnupperkurspreis': taster_course_price,
    'Ratingscore': ratingscore,
    'Reviewanzahl': ratingcount,
     'Rating Faktoren': [ratingfactors],
    'E-Mail': contact_row[0],
    'Homepage': contact_row[1],
    'Telefon': contact_row[2],
    'Eversports Pole Studio Seite': url

}
# create pandas dataframe
pole_studio_overview_df = pd.DataFrame.from_dict(pole_studio_overview)

# Make a datastamp
pole_studio_overview_df['Version'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# display dataframe
pole_studio_overview_df


,PoleStudio,PLZ,Stadt,Adresse,Pole Studio Beschreibung,Start Preis,Schnupperkurspreis,Ratingscore,Reviewanzahl,Rating Faktoren,E-Mail,Homepage,Telefon,Eversports Pole Studio Seite,Version
0,Yoga and Pole Art by Selina,53783,Zum Wingert 10,53783 Eitorf,Yoga and Pole Art by Selina bietet dir Online ...,"24,00 €","24,00 €",5.0,49 Rezensionen,"[Raumqualität 5.0, Freundlichkeit & Service 5....",yogaandpoleartbyselina@gmail.com,http://www.yogaandpoleartbyselina.com,+49 15738925807,https://www.eversports.de/s/yoga-and-pole-art-...,2023-04-18 00:08:34


In [118]:
pole_studio_overview_df = pole_studio_overview_df.drop(["Rating Faktoren"], axis=1)
# pole_studio_overview_df

In [119]:
# pole_studio_overview_df.to_csv('pole_studio_overview.csv', index=False)

# create the "CSV" directory if it does not exist
# if not os.path.exists("PoleStudio_CSV"):
     # os.makedirs("PoleStudio_CSV")

# save the CSV file in the "CSV" directory
pole_studio_overview_df.to_csv(f"{pole_studio_name}.csv", index=False)

Prepare a SQL Database for the Data

In [120]:
# SQL Setup

# host = 'Your information!'
# schema = 'hallofpole'
# user = 'Your information!'
# password = 'Your information!'
# port = Your information!

# # create connection string with the database name included
# con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# # create sqlalchemy engine
# engine = create_engine(con)


In [121]:
# # Write the DataFrame to the SQL database
# pole_studio_overview_df.to_sql(name='pole_studio_overview', con=con, if_exists='replace', index=False)


In [122]:
# #Read SQL and create a CSV

# # create sqlalchemy engine
# engine = create_engine(con)

# # read table from SQL database into a DataFrame
# pole_studio_overview_df = pd.read_sql_table('pole_studio_overview', con=engine)

# # write DataFrame to CSV file
# pole_studio_overview_df.to_csv('pole_studio_overview.csv', index=False)
